In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import folium

In [3]:
Covid19 = pd.read_json('../input/json-file/india_statewise.json')
Covid19.head()

,success,data,lastRefreshed,lastOriginUpdate
lastRefreshed,True,2020-07-12T03:54:15.207Z,2020-07-12T03:54:15.207Z,2020-07-12T03:54:15.207Z
source,True,covid19india.org,2020-07-12T03:54:15.207Z,2020-07-12T03:54:15.207Z
statewise,True,"[{'state': 'Maharashtra', 'confirmed': 246600,...",2020-07-12T03:54:15.207Z,2020-07-12T03:54:15.207Z
total,True,"{'confirmed': 850358, 'recovered': 536231, 'de...",2020-07-12T03:54:15.207Z,2020-07-12T03:54:15.207Z


#  Normalizing JSON File

In [4]:
df_india = pd.io.json.json_normalize(Covid19['data']['statewise'])
df_india

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,state,confirmed,recovered,deaths,active
0,Maharashtra,246600,136985,10116,99202
1,Tamil Nadu,134226,85915,1898,46413
2,Delhi,110921,87692,3334,19895
3,Gujarat,41027,28685,2033,10309
4,Uttar Pradesh,35092,22689,913,11490
5,Karnataka,36216,14718,615,20879
6,Telangana,33402,20919,348,12135
7,West Bengal,28453,17959,906,9588
8,Andhra Pradesh,27235,14393,309,12533
9,Rajasthan,23748,17869,503,5376


In [5]:
df_india.set_index('state',inplace=True)
df_india.head()

,confirmed,recovered,deaths,active
state,,,,
Maharashtra,246600,136985,10116,99202
Tamil Nadu,134226,85915,1898,46413
Delhi,110921,87692,3334,19895
Gujarat,41027,28685,2033,10309
Uttar Pradesh,35092,22689,913,11490


In [6]:
df_india.drop('State Unassigned',inplace=True)

In [7]:
locations = {
    "Kerala" : [10.8505,76.2711],
    "Maharashtra" : [19.7515,75.7139],
    "Karnataka": [15.3173,75.7139],
    "Telangana": [18.1124,79.0193],
    "Uttar Pradesh": [26.8467,80.9462],
    "Rajasthan": [27.0238,74.2179],
    "Gujarat":[22.2587,71.1924],
    "Delhi" : [28.7041,77.1025],
    "Punjab":[31.1471,75.3412],
    "Tamil Nadu": [11.1271,78.6569],
    "Haryana": [29.0588,76.0856],
    "Madhya Pradesh":[22.9734,78.6569],
    "Jammu and Kashmir":[33.7782,76.5762],
    "Ladakh": [34.1526,77.5770],
    "Andhra Pradesh":[15.9129,79.7400],
    "West Bengal": [22.9868,87.8550],
    "Bihar": [25.0961,85.3131],
    "Chhattisgarh":[21.2787,81.8661],
    "Chandigarh":[30.7333,76.7794],
    "Uttarakhand":[30.0668,79.0193],
    "Himachal Pradesh":[31.1048,77.1734],
    "Goa": [15.2993,74.1240],
    "Odisha":[20.9517,85.0985],
    "Andaman and Nicobar Islands": [11.7401,92.6586],
    "Puducherry":[11.9416,79.8083],
    "Manipur":[24.6637,93.9063],
    "Mizoram":[23.1645,92.9376],
    "Assam":[26.2006,92.9376],
    "Meghalaya":[25.4670,91.3662],
    "Tripura":[23.9408,91.9882],
    "Arunachal Pradesh":[28.2180,94.7278],
    "Jharkhand" : [23.6102,85.2799],
    "Nagaland": [26.1584,94.5624],
    "Sikkim": [27.5330,88.5122],
    "Dadra and Nagar Haveli and Daman and Diu":[20.1809,73.0169],
    "Lakshadweep":[10.5667,72.6417],
}

In [8]:
df_india['Lat'] = ""
df_india['Long'] = ""
for index in df_india.index:
    df_india.loc[df_india.index==index,'Lat'] = locations[index][0]
    df_india.loc[df_india.index==index,'Long'] = locations[index][1]

In [9]:
df_india.head()

,confirmed,recovered,deaths,active,Lat,Long
state,,,,,,
Maharashtra,246600,136985,10116,99202,19.7515,75.7139
Tamil Nadu,134226,85915,1898,46413,11.1271,78.6569
Delhi,110921,87692,3334,19895,28.7041,77.1025
Gujarat,41027,28685,2033,10309,22.2587,71.1924
Uttar Pradesh,35092,22689,913,11490,26.8467,80.9462


In [10]:
india = folium.Map(location=[23,80],tiles='cartodbpositron',zoom_start=4,max_zoom=7,min_zoom=4,height=500,width='80%')

In [11]:
india

# Displaying Folium Map using marker and passing values to it

In [12]:
for i in range(0,len(df_india[df_india['confirmed']>0].index)):
    
    folium.Circle(    
            location=[df_india.iloc[i]['Lat'],df_india.iloc[i]['Long']],
            tooltip="<h5>"+df_india.iloc[i].name+"</h5>"+
            "<li>Confirmed:"+str(df_india.iloc[i]['confirmed'])+"</li>"+
            "<li>Recovered:"+str(df_india.iloc[i]['recovered'])+"</li>"+
            "<li>Deaths:"+str(df_india.iloc[i]['deaths'])+"</li>"+
            "<li>Active:"+str(df_india.iloc[i]['active'])+"</li>",

            radius=(int((df_india.iloc[i]['confirmed']))*0.5),
            color = '#ff6600',
            fill_color = '#ff6600',
            fill=True).add_to(india)
india

In [13]:
len(df_india[df_india['confirmed']>0].index)

35